# Optimizing Model Parameters

* Now that we have a model and data it’s time to train, validate and test our model by optimizing its parameters on our data. Training a model is an iterative process; in each iteration the model makes a guess about the output, calculates the error in its guess (*loss*), collects the derivatives of the error with respect to its parameters (as we saw in the previous section), and **optimizes** these parameters using gradient descent. For a more detailed walkthrough of this process, check out this video on [backpropagation from 3Blue1Brown](https://www.youtube.com/watch?v=tIeHLnjs5U8).

## Prerequisite Code

* We load the code from the previous sections on [Datasets & DataLoaders](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html) and [Build Model](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html).

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
        
model = NeuralNetwork()

100%|██████████| 26421880/26421880 [00:01<00:00, 17146370.16it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 298140.00it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5446184.44it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 15818517.94it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



## Hyperparameters

* Hyperparameters are adjustable parameters that let you control the model optimization process. Different hyperparameter values can impact model training and convergence rates ([read more](https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html) about hyperparameter tuning)

* We define the following hyperparameters for training:
    - **Number of Epochs** - the number times to iterate over the dataset
    - **Batch Size** - the number of data samples propagated through the network before the parameters are updated
    - **Learning Rate** - how much to update models parameters at each batch/epoch. Smaller values yield slow learning speed, while large values may result in unpredictable behavior during training.

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## Optimization Loop

* Once we set our hyperparameters, we can then train and optimize our model with an optimization loop. Each iteration of the optimization loop is called an **epoch**.

* Each epoch consists of two main parts:
    - **The Train Loop** - iterate over the training dataset and try to converge to optimal parameters.
    - **The Validation/Test Loop** - iterate over the test dataset to check if model performance is improving.

* Let’s briefly familiarize ourselves with some of the concepts used in the training loop. Jump ahead to see the [Full Implementation](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html#full-impl-label) of the optimization loop.

## Loss Function

* When presented with some training data, our untrained network is likely not to give the correct answer. **Loss function** measures the degree of dissimilarity of obtained result to the target value, and it is the loss function that we want to minimize during training. To calculate the loss we make a prediction using the inputs of our given data sample and compare it against the true data label value.

* Common loss functions include [nn.MSELoss](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss) (Mean Square Error) for regression tasks, and [nn.NLLLoss](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html#torch.nn.NLLLoss) (Negative Log Likelihood) for classification. [nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss) combines nn.LogSoftmax and nn.NLLLoss.

* We pass our model’s output logits to `nn.CrossEntropyLoss`, which will normalize the logits and compute the prediction error.

In [3]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

## Optimizer

* Optimization is the process of adjusting model parameters to reduce model error in each training step. `Optimization algorithms` define how this process is performed (in this example we use Stochastic Gradient Descent). All optimization logic is encapsulated in the optimizer object. Here, we use the SGD optimizer; additionally, there are many [different optimizers](https://pytorch.org/docs/stable/optim.html) available in PyTorch such as ADAM and RMSProp, that work better for different kinds of models and data.

* We initialize the optimizer by registering the model's parameters that need to be trained, and passing in the learning rate hyperparameter.

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

* Inside the training loop, optimization happens in three steps:
    - `Call optimizer.zero_grad()` to reset the gradients of model parameters. Gradients by default add up; to prevent double-counting, we explicitly zero them at each iteration.
    - Backpropagate the prediction loss with a call to `loss.backward()`. PyTorch deposits the gradients of the loss w.r.t. each parameter.
    - Once we have our gradients, we call `optimizer.step()` to adjust the parameters by the gradients collected in the backward pass.

## Full Implementation

* We define `train_loop` that loops over our optimization code, and `test_loop` that evaluates the model's performance against our `test data`

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # set the model to training mode - important for batch normalization and dropout layers
    # unnecessary in this situation but add for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d} / {size:>5d}]")
            
def test_loop(dataloader, model, loss_fn):
    # set the model to evaluation mode - important for batch normalization and dropout layers
    # unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    # evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")

* We initialize the loss function and optimizer, and pass it to `train_loop` and `test_loop`. Feel free to increase the number of epochs to track the model’s improving performance.

In [6]:
import tqdm

from tqdm.notebook import tqdm

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in tqdm(range(epochs)):
    print(f"Epoch {t + 1}\n---------------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done")

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1
---------------------------------------
loss: 2.303916 [   64 / 60000]
loss: 2.290578 [ 6464 / 60000]
loss: 2.273265 [12864 / 60000]
loss: 2.271098 [19264 / 60000]
loss: 2.242660 [25664 / 60000]
loss: 2.219155 [32064 / 60000]
loss: 2.225608 [38464 / 60000]
loss: 2.188614 [44864 / 60000]
loss: 2.189776 [51264 / 60000]
loss: 2.155869 [57664 / 60000]
Test Error: 
 Accuracy: 44.5%, Avg loss: 2.151989

Epoch 2
---------------------------------------
loss: 2.160142 [   64 / 60000]
loss: 2.154176 [ 6464 / 60000]
loss: 2.097667 [12864 / 60000]
loss: 2.117087 [19264 / 60000]
loss: 2.057376 [25664 / 60000]
loss: 1.997816 [32064 / 60000]
loss: 2.026666 [38464 / 60000]
loss: 1.946364 [44864 / 60000]
loss: 1.949549 [51264 / 60000]
loss: 1.874892 [57664 / 60000]
Test Error: 
 Accuracy: 53.2%, Avg loss: 1.880368

Epoch 3
---------------------------------------
loss: 1.908070 [   64 / 60000]
loss: 1.884996 [ 6464 / 60000]
loss: 1.772500 [12864 / 60000]
loss: 1.811813 [19264 / 60000]
loss: 1.69